In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.applications import ResNet50

In [3]:
train_data = ImageDataGenerator()
test_data = ImageDataGenerator()

In [4]:
img_size=(224,224)
batch_size = 32

In [5]:
res=ResNet50(weights='imagenet',include_top='False',input_shape=(224,224,3))


102967424/102967424 [==============================] - 4s 0us/step


In [6]:
for layer in res.layers:
  layer.trainable=False

In [7]:
x = Flatten()(res.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [8]:
sr_train_dir = '/content/drive/MyDrive/chest_xray/train'
sr_val_dir='/content/drive/MyDrive/chest_xray/val'
sr_test_dir='/content/drive/MyDrive/chest_xray/test'

In [9]:
sr_train_generator = train_data.flow_from_directory(
    sr_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Found 5216 images belonging to 2 classes.


In [10]:
sr_test_generator = test_data.flow_from_directory(
    sr_test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL','PNEUMONIA'],
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [11]:
sr_val_generator = test_data.flow_from_directory(
    sr_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Found 16 images belonging to 2 classes.


In [12]:
model_sr = Model(inputs=res.input, outputs=predictions)


In [13]:
model_sr.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
history_sr = model_sr.fit(sr_train_generator, validation_data=sr_val_generator, epochs=10)


Epoch 1/10
163/163 [==============================] - 1144s 6s/step - loss: 0.4743 - accuracy: 0.7799 - val_loss: 0.6215 - val_accuracy: 0.4375
Epoch 2/10
163/163 [==============================] - 73s 444ms/step - loss: 0.3028 - accuracy: 0.8758 - val_loss: 0.5467 - val_accuracy: 0.5000
Epoch 3/10
163/163 [==============================] - 72s 439ms/step - loss: 0.2652 - accuracy: 0.8873 - val_loss: 0.5447 - val_accuracy: 0.6875
Epoch 4/10
163/163 [==============================] - 79s 482ms/step - loss: 0.2466 - accuracy: 0.8949 - val_loss: 0.5424 - val_accuracy: 0.6875
Epoch 5/10
163/163 [==============================] - 79s 481ms/step - loss: 0.2325 - accuracy: 0.8984 - val_loss: 0.5589 - val_accuracy: 0.6875
Epoch 6/10
163/163 [==============================] - 73s 446ms/step - loss: 0.2227 - accuracy: 0.9018 - val_loss: 0.6874 - val_accuracy: 0.6875
Epoch 7/10
163/163 [==============================] - 70s 430ms/step - loss: 0.2146 - accuracy: 0.9070 - val_loss: 0.6868 - val_acc

In [18]:
test_loss_1, test_acc_1 = model_sr.evaluate(sr_test_generator)
print("Test Accuracy:", test_acc_1)

20/20 [==============================] - 7s 337ms/step - loss: 0.5463 - accuracy: 0.7628
Test Accuracy: 0.7628205418586731


In [15]:
model_sr.save("resnet_pneumonia_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
